In [1]:
import os
import json
import re
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
def load_results(path):
    experiment_file = os.path.join(path, "experiments.json")
    
    with open(experiment_file) as train_file:
        dict_train = json.load(train_file)
        experiments = json_normalize(dict_train)
    results = pd.DataFrame()
    
    for subdir, dirs, files in os.walk(os.path.join(path, "results")):
        for file in files:
            f = os.path.join(subdir, file)
            with open(f) as res_file:
                res = json.load(res_file)
                results = results.append(json_normalize(res), sort=True)
                
    results.hash = results.hash.astype(str)
    experiments.hash = experiments.hash.astype(str)
    
    e = experiments.drop(['quality_metrics.kappa'], axis=1).set_index('hash') # duplicate entry
    r = results[['hash', 'method', 'quality_metrics.kappa', 'result.C', 'result.gamma', 'result.metrics.kappa', 'result.metrics.f1']].set_index('hash')
    res_df = r.join(e)
    res_df['run'] = path.split("/")[-1]
    return res_df

In [3]:
res_AL = load_results("../data/output/AL")
res_competitors = load_results("../data/output/Competitors")

In [4]:
res_full = pd.concat([res_AL, res_competitors], axis=0, sort=False)

In [5]:
res_full.to_pickle("../data/output/run_full.pkl")

In [6]:
res_kappa = load_results("../data/output/AL-kappa")
res_kappa.to_pickle("../data/output/run_kappa.pkl")